In [75]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

df1 = pd.read_csv("NSW_Road_Crash.csv")
df1.dropna()
df1 = df1.rename(columns={'Direction': 'cardinal_direction_name'})
df1 = df1.rename(columns={'Year of crash': 'year'})

df2 = pd.read_csv("Traffic_Volume_Viewer.csv")
mask = df2['classification_type'].isin(['LIGHT VEHICLES', 'HEAVY VEHICLES']) & ~df2['cardinal_direction_name'].str.contains('AND|BOTH') & ~df2['period'].isin(['ALL DAYS', 'WEEKDAYS','PUBLIC HOLIDAYS']) & (df2['year'] >= 2017)
df2_new = df2[mask]

df2.dropna()
df2_new.head()

names = df2_new['period'].unique()
print(names)

['AM PEAK' 'OFF PEAK' 'PM PEAK' 'WEEKENDS']


In [76]:
#function to change String format
def String_format(name):
    if isinstance(name, str):
        new_str = name.capitalize()
        return new_str

String_format("MUAZ")

def abreviation(word,l1,l2):
    #search for the word in l1 and return i index of l2
    i = 0
    while i < len(l1):
       if word == l1[i]:
           return l2[i]
       else:
           i += 1
           continue 

In [77]:
# Street Definitions
unique_street_types = df1['Street type'].unique()
unique_street_types_list = list(unique_street_types)
full_street_type = ['Road','Way','Highway','Drive','Parade','Street','Avenue','Expressway','0','Distributor','Parkway','Court','Lane','Place','Boulevard','Close','Circuit','Crescent','Terrace','Bypass','Mall','Trail','Promenade','Grove','Track','Square','Esplanade','Gardens','Glen','Route','Driveway','Causeway','Loop','Chase','Access','Row','Walk','0']

# Time Definitions 
unique_times = df1['Two-hour intervals'].unique()
unique_times_list = list(unique_times)
time_periods = ['PM PEAK','OFF PEAK','PM PEAK','OFF PEAK','OFF-PEAK','OFF-PEAK','AM PEAK','AM PEAK','PM PEAK','OFF PEAK','OFF PEAK','OFF PEAK','0','0']

valid_directions = ['NORTH', 'SOUTH', 'EAST', 'WEST']
df1["cardinal_direction_name"] = df1["cardinal_direction_name"].str.upper()

#Loop through the dataset
l1 = unique_street_types
l2 = full_street_type

formatted_rows = []
day_period = []
cardinal_direction_name = [] 

# CRASH DATASET
for index, row in df1.iterrows():
    if not pd.isnull(row["Street Crash"]) and not pd.isnull(row["Street type"]):
        formatted_string = String_format(row["Street Crash"]) + " " + abreviation(row["Street type"], l1, l2)
        formatted_rows.append(formatted_string)
    else:
        formatted_rows.append(None)
    
    if row['Day of week of crash'] in ['Saturday','Sunday']: 
        day_period_string = 'WEEKENDS'
        
    else: 
        day_period_string = abreviation(row["Two-hour intervals"], unique_times_list, time_periods)
        
    day_period.append(day_period_string)
    
df1["road_name"] = formatted_rows
df1["period"] = day_period

df1 = df1[df1['cardinal_direction_name'].isin(valid_directions)]

In [78]:
# Filter both crash and traffic volume dataframe based on Road Name Column
df_crash = df1[df1['road_name'].isin(df2_new['road_name']) & 
               df1['year'].isin(df2_new['year']) &
               df1['period'].isin(df2_new['period']) & 
               df1['cardinal_direction_name'].isin(df2_new['cardinal_direction_name'])]

df_traffic = df2_new[df2_new['road_name'].isin(df1['road_name']) & 
                     df2_new['year'].isin(df1['year']) &
                     df2_new['period'].isin(df1['period']) & 
                     df2_new['cardinal_direction_name'].isin(df1['cardinal_direction_name'])]
                            

print('Crash Data:',df1.shape[0])
print('Crash Data Filtered:',df_crash.shape[0])

print('Traffic Data:',df2_new.shape[0])
print('Traffic Data Filtered:',df_traffic.shape[0])

print(df_traffic['year'].unique())

Crash Data: 46294
Crash Data Filtered: 7569
Traffic Data: 16021
Traffic Data Filtered: 8080
[2017 2018 2019 2020 2021]


In [84]:
import time

start_time = time.time()

# Loop through each element in df_traffic
crash_count_list = []
for index, row in df_traffic.iterrows():
    # Filter crash dataframe to only include entries that allign with criteria
    crash_temp = df_crash[(df_crash['road_name'] == row['road_name']) & 
                          (df_crash['year'] == row['year']) &
                          (df_crash['period'] == row['period']) &
                          (df_crash['cardinal_direction_name'] == row['cardinal_direction_name'])]
    # Get number of entires and store as new list
    crash_count = crash_temp.shape[0]
    crash_count_list.append(crash_count)

df_traffic['crash_count'] = crash_count_list
df_traffic.head()

end_time = time.time()

time_taken = end_time - start_time
print('Time Taken:',time_taken)

print(df_traffic['crash_count'].unique())

Time Taken: 9.169407606124878
[ 0  1 20 19 35 36  2  3  5  4  6 12  7 23 29 45 58 49 56  8 14  9 25 50
 46 54 44 24 17 41 40 48 21 27 51 47 60 13 38 34 32 37 33 26 30 16 15 10
 11 18 28 22]


In [82]:
df_traffic.head(60)

,the_geom,station_id,road_name,suburb,cardinal_direction_name,classification_type,year,period,traffic_count,wgs84_latitude,wgs84_longitude,crash_count
41308,0101000020E61000004C16F71F99DD6240FA7B293C68F0...,100001,Cambridge Street,Canley Heights,NORTH,LIGHT VEHICLES,2017,AM PEAK,5398,-33.878181,150.924942,0
41309,0101000020E61000004C16F71F99DD6240FA7B293C68F0...,100001,Cambridge Street,Canley Heights,NORTH,HEAVY VEHICLES,2017,AM PEAK,617,-33.878181,150.924942,0
41311,0101000020E61000004C16F71F99DD6240FA7B293C68F0...,100001,Cambridge Street,Canley Heights,SOUTH,LIGHT VEHICLES,2017,AM PEAK,3814,-33.878181,150.924942,0
41312,0101000020E61000004C16F71F99DD6240FA7B293C68F0...,100001,Cambridge Street,Canley Heights,SOUTH,HEAVY VEHICLES,2017,AM PEAK,521,-33.878181,150.924942,0
41317,0101000020E61000004C16F71F99DD6240FA7B293C68F0...,100001,Cambridge Street,Canley Heights,NORTH,LIGHT VEHICLES,2017,OFF PEAK,10158,-33.878181,150.924942,0
41318,0101000020E61000004C16F71F99DD6240FA7B293C68F0...,100001,Cambridge Street,Canley Heights,NORTH,HEAVY VEHICLES,2017,OFF PEAK,1086,-33.878181,150.924942,0
41320,0101000020E61000004C16F71F99DD6240FA7B293C68F0...,100001,Cambridge Street,Canley Heights,SOUTH,LIGHT VEHICLES,2017,OFF PEAK,9630,-33.878181,150.924942,0
41321,0101000020E61000004C16F71F99DD6240FA7B293C68F0...,100001,Cambridge Street,Canley Heights,SOUTH,HEAVY VEHICLES,2017,OFF PEAK,1091,-33.878181,150.924942,0
41326,0101000020E61000004C16F71F99DD6240FA7B293C68F0...,100001,Cambridge Street,Canley Heights,NORTH,LIGHT VEHICLES,2017,PM PEAK,4678,-33.878181,150.924942,0
41327,0101000020E61000004C16F71F99DD6240FA7B293C68F0...,100001,Cambridge Street,Canley Heights,NORTH,HEAVY VEHICLES,2017,PM PEAK,329,-33.878181,150.924942,0
